<a href="https://colab.research.google.com/github/rescobarcorrea/ArcGIS_DB_Managment/blob/main/List_GDB_domains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code to list features and associated domains in a geo data base with arc py and arcgis pro

In [ ]:
#code to read GDB  featureclassses and its domains

#Imports
import arcpy
import os
import csv



In [ ]:
#inputs, the path to the gdb conection and the location to save the csv
#crate a gdb conection in your workspace and the get its path and paste it here:
arcpy.env.workspace = r"D:\Users\Rogger.Correa\AppData\Roaming\Esri\ArcGISPro\Favorites\GISM@awsgisdb.sde"
csv_path=r"D:\Users\Rogger.Correa\Downloads\gdbstructure.csv"

In [ ]:
#read datasets inside the gdb
datasets = arcpy.ListDatasets(feature_type='feature')
datasets = [''] + datasets if datasets is not None else []
header=['Featureclass','FieldName','FieldType','Fieldlenght','FieldDomain']

with open(csv_path,'w') as csv_file:
    print('header')
    w=csv.writer(csv_file)
    w.writerow(header)
    for ds in datasets:
        print('##'+ds)
        for fc in arcpy.ListFeatureClasses(feature_dataset=ds):
            fcpath = os.path.join(arcpy.env.workspace, ds, fc)
            print('-'+fc)
            fields=arcpy.ListFields(fcpath)
            for field in fields:
                #print("--{0},{1},{2},{3}"
                 #.format(field.name, field.type, field.length,field.domain))
                w.writerow((fc,field.name, field.type, field.length,field.domain))
csv_file.close
print('**********************')
print('**********************')
print('********done**********')
print('**********************')
        

## this is the formay to find it later down    
#print(gdb_domains[gdb_domain_names=='DmGeometryType'].domainType)
#print(gdb_domains[gdb_domain_names=='DmGeometryType'].name)

In [ ]:
#MAIN CODE, READS DOMAINS, SUBTYPES AND FIND OUT UNUSED DOMAINS
#read datasets inside the gdb
datasets = arcpy.ListDatasets(feature_type='feature')
datasets = [''] + datasets if datasets is not None else []
header=['Featureclass','SubType','DefiningSTField','DefiningValue','FieldName','DefaultValue','FieldDomain']

# get all the GDB domains
gdb_domains=arcpy.da.ListDomains(arcpy.env.workspace)
gdb_domain_names=[]
gdb_used_domains=[]
gdb_not_used_domains=[]

with open(csv_path,'w', newline='') as csv_file:
    print(header)
    w=csv.writer(csv_file)
    w.writerow(header)
    for ds in datasets:
        print('##'+ds)
        for fc in arcpy.ListFeatureClasses(feature_dataset=ds):
            fcpath = os.path.join(arcpy.env.workspace, ds, fc)
            
            #grant permisions
            try:
                arcpy.ChangePrivileges_management(fcpath, "sde", "GRANT", "GRANT")
                arcpy.ChangePrivileges_management(fcpath, "FNSW", "GRANT", "GRANT")
                arcpy.ChangePrivileges_management(fcpath, "DBO", "GRANT", "GRANT")
            except:
                pass
            print('-'+fc)
            subtypes=arcpy.da.ListSubtypes(fcpath)
            print('..')
            for st in subtypes:
                stname=subtypes[st]['Name']
                deffield=subtypes[st]['SubtypeField']
                #print (stname)
                fields=subtypes[st]['FieldValues']
                for field, fieldvals in list(fields.items()):
                    if stname.lower()==fc.lower():
                        stname=''
                        print('.')
                    if not fieldvals[1] is None:
                        #print('-subtype:{0} defvalue:{1} DefiningField: {2}-Field:{3} -DefaultValue:{4} -Domain:{5}'
                            #.format(stname,st,deffield,field, fieldvals[0],fieldvals[1].name))
                        w.writerow((fc,stname,deffield,st,field, fieldvals[0],fieldvals[1].name))
                        gdb_used_domains.append(fieldvals[1].name)
                        #print('...')
                    else:
                        w.writerow((fc,stname,deffield,st,field, fieldvals[0],fieldvals[1]))
                        #print('.....')
                        #print('-subtype:{0} defvalue:{1} DefiningField: {2}-Field:{3} -DefaultValue:{4} -Domain:{5}'
                            #.format(stname,deffield,st,field, fieldvals[0],fieldvals[1])            



csv_file.close
print('**********************')
print('**********************')
print('********done**********')
print('**********************')
print('**********************')
        
## this is the formay to find it later down    
#print(gdb_domains[gdb_domain_names=='DmGeometryType'].domainType)
#print(gdb_domains[gdb_domain_names=='DmGeometryType'].name)

In [ ]:
#this was for somethig else, now not necessary I think
# you have to read all the domains of the GDB first, then you can find the domains properties later down
csv_logpath=r"D:\Users\Rogger.Correa\Downloads\gdbalterationlogGISR.csv"
header2=['changes','changetype','d.name','d.owner','d.type','d.domainType','d.splitPolicy']
gdb_not_used_domains=[]

with open(csv_logpath,'w', newline='') as csv_logfile:
    print(header)
    w=csv.writer(csv_logfile)
    w.writerow(header2)    
    
    arcpy.DisconnectUser(arcpy.env.workspace, 'ALL')
    counter=0
    for d in gdb_domains:
            counter+=1
            gdb_domain_names.append(d.name)
            #print('.')
            print(d.name)
            if counter%100==0: # every 30 refresh the disconection of the WS
                arcpy.DisconnectUser(arcpy.env.workspace, 'ALL')
                print('WS disconections refreshed- ',counter)
            try:
                arcpy.management.AlterDomain(arcpy.env.workspace, d.name, d.name, d.description, "DUPLICATE", "DEFAULT")
                #print('splitpolicy','changed',d.name,d.owner,d.type,d.domainType)
                w.writerow(('splitpolicy','changed',d.name,d.owner,d.type,d.domainType,d.splitPolicy))
                #arcpy.DisconnectUser(arcpy.env.workspace, 'ALL')
            except:
                #print('splitpolicy','not_changed',d.name,d.owner,d.type,d.domainType)
                w.writerow(('splitpolicy','not_changed',d.name,d.owner,d.type,d.domainType,d.splitPolicy))
                pass            
            #This bit deletes un-used domains

            if d.name not in gdb_used_domains:
                #print('notused',d.name)
                w.writerow(('not used','deleted',d.name,d.owner,d.type,d.domainType))
                gdb_not_used_domains.append(d.name)
                #delete domains
                try:
                    arcpy.DeleteDomain_management(arcpy.env.workspace, d.name)
                except:
                    #print(d.name, 'could not be deleted')
                    w.writerow(('not used','not_deleted',d.name,d.owner,d.type,d.domainType))
                    pass                   
csv_logfile.close
print('**********************')
print('**********************')
print('********done**********')
print('**********************')
print('**********************')